In [23]:
import pandas as pd
from nltk.corpus import sentiwordnet as swn
import nltk
import pandas as pd
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict

In [24]:
input_df = pd.read_excel('Input/Review_Input.xlsx')

C:\Users\utkar\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [25]:
input_df

,id,Reviews
0,1,The familial bond remains stronger than ever i...
1,2,A big and bubbly adventure overflowing with wi...
2,3,Shazam 2: The Fury of Gods is more like a 90s ...
3,4,"In a drama this dumb and unaware, the buffed-u..."
4,5,"Unfortunately, the new sequel proves an adage:..."
5,6,Too derivative from previous superhero films a...
6,7,[Shazam! Fury of the Gods] strains too hard fo...
7,8,The charm of this movie relies heavily on Levi...
8,9,The filmmakers have clearly asked themselves h...
9,10,The best thing about Shazam! Fury of the Gods ...


In [29]:
# Load positive and negative words
with open('MasterDictionary/positive_words.txt', 'r') as file:
    positive_words = set(line.strip() for line in file)
with open('MasterDictionary/negative_words.txt', 'r') as file:
    negative_words = set(line.strip() for line in file)

# Calculate sentiment scores for each review
for index, row in input_df.iterrows():
    Review_text = row['Reviews']

    tokens = nltk.word_tokenize(Review_text)

    positive_score = (sum(1 for token in tokens if token in positive_words) / len(tokens)) * 100

    negative_score = (sum(1 for token in tokens if token in negative_words) / len(tokens)) * -100

    # Calculate intensity of sentiment for each word in the review
    intensity_score = 0
    for token in tokens:
        senti_synsets = list(swn.senti_synsets(token))  # convert filter object to list
        if senti_synsets:
            # Consider only the first synset for simplicity
            senti_synset = senti_synsets[0]
            intensity_score += (senti_synset.pos_score() - senti_synset.neg_score())

    # Normalize the intensity score by the total number of words in the review
    intensity_score /= (len(tokens) + 0.000001)

    # Determine the sentiment category based on intensity score
    if intensity_score >= 0.5:
        sentiment_category = 'Very Positive'
    elif intensity_score > 0 and intensity_score < 0.5:
        sentiment_category = 'Positive'
    elif intensity_score == 0:
        sentiment_category = 'Neutral'
    elif intensity_score < 0 and intensity_score > -0.5:
        sentiment_category = 'Negative'
    else:
        sentiment_category = 'Very Negative'
        
    # Determine the emotion based on intensity score
    if intensity_score >= 0.75:
        emotion = 'joy'
    elif intensity_score >= 0.25 and intensity_score < 0.75:
        emotion = 'happy'
    elif intensity_score >= -0.25 and intensity_score < 0.25:
        emotion = 'neutral'
    elif intensity_score >= -0.75 and intensity_score < -0.25:
        emotion = 'sad'
    else:
        emotion = 'anger'

    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

    total_words = len(tokens)

    subjectivity_score = (positive_score + abs(negative_score)) / (total_words + 0.000001)

    # Assign sentiment based on polarity score
    if polarity_score > 0:
        sentiment = 'positive'
    elif polarity_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    # Add new columns to dataframe
    input_df.at[index, 'POSITIVE SCORE(%)'] = positive_score
    input_df.at[index, 'NEGATIVE SCORE(%)'] = negative_score
    input_df.at[index, 'INTENSITY SCORE'] = emotion
    input_df.at[index, 'SENTIMENT'] = sentiment
    input_df.at[index, 'SENTIMENT CATEGORY'] = sentiment_category
    input_df.at[index, 'EMOTION'] = emotion
input_df.to_excel('Output/Movie_Review_Sentimental_Analysis_Report.xlsx', index=False)
